# **I/O 효율화 튜닝의 중요성**

- 디스크를 경유한 입출력은 메모리를 통한 입출력보다 평균적으로 10,000배 이상 느림.
- 데이터베이스는 하나의 파일 시스템.
    - SQL을 이용하여 데이터 파일에 데이터를 저장하고 출력시 DB 버퍼 캐쉬에 데이터가 없을 경우 디스크를  통하여 데이터를 버퍼 캐쉬에 저장한 후 결과를 줌.
- 물리적 디스크 I/O가 필요할 경우 서버 프로세스가 디스크에서 원하는 블럭을 가져오고, 같은 데이터를 원할 경우 경합이 발생하여 대기 시간이 길어짐.
- 모든 자료를 메모리에 저장하여 사용하면 좋지만 비용과 기술적 측면에서 한계.
- 시중에는 메모리 데이터베이스가 사용되지만 극히 빠른 액세스가 제한되는 업무 영업에 국한되어 사용되어지고 있음.
- 유한한 메모리 자원을 효율적으로 사용해야 하므로 자주 액세스하는 블록들이 캐시에 더 오래 남아 있도록 LRU(Least Recently Used) 알고리즘을 사용.

> **디스크 I/O를 최소화하고, 대부분의 처리를 메모리에서 할 수 있도록 버퍼 캐시 효율성을 높이는 것이 성능 튜닝의 핵심**


# **버퍼 캐시 히트율(Buffer Cache Hit Ratio)**

- 읽은 블록 중에서 얼마만큼을 메모리 버퍼 캐시에서 찾았는지를 나타내는 것.

> 버퍼 캐시 히트율 공식 (BCHR)
>
> = (캐시에서 곧바로 찾은 블록 수 / 총 읽은 블록 수)100
>
> = ((논리적 블록 읽기 - 물리적 블록 읽기) 논리적 블록읽기) 100
>
> = (1 - (물리적 블록 읽기) / (논리적 블록읽기) * 100)

> - '논리적 블록읽기' = '총 읽은 블록 수'
> - '논리적 블록읽기' = SQL을 수행하는 동안 캐시에서 읽은 총 블록 수
> - '캐시에서 곧바로 찾은 블록 수' = '논리적 블록읽기' - '물리적 블록 읽기'

- Direct Path Read 방식을 제외한 경우 모든 블록 읽기는 버퍼 캐시를 통해 이뤄짐.
- 읽고자 하는 블록을 먼저 버퍼 캐시에서 찾아보고 없을 때 디스크에서 읽으며, 이 경유도 디스크로부터 곧바로 읽는게 아니라 버퍼 캐시에 적재한 후 읽음

**[실행계획 예시]**


```sql
================================================================================
call     count       cpu    elapsed       disk      query    current        rows
------- ------  -------- ---------- ---------- ---------- ----------  ----------
Parse        1      0.00       0.00          0          0          0           0
Execute      1      0.00       0.00          0          0          0           0
Fetch        2      0.00       0.00          0        178          0           1
------- ------  -------- ---------- ---------- ---------- ----------  ----------
total        4      0.00       0.00          0        178          0           1
================================================================================
```

- Disk 항목이 물리적 블록읽기에 해당.
- 논리적 블록읽기는 Query와 Current 항목을 더해서 구함.
- Direct Path Read 방식으로 읽은 블록이 없으면 이 두 값을 더한 것이 총 읽은 블록 수가 됨.
- 총 1,351,677개 블록을 읽었는데, 그 중 601,458개는 디스크에서 버퍼캐시로 적재한 후에 읽음. 즉 BCHR은 56%.
- BCHR = (1 - (Disk / (Query + Current))) x 100
- BCHR = (1 - (601,458 / (1,351,677 + 0))) x 100 = 55.5%

> **논리적인 블록 요청 횟수를 줄이고, 물리적으로 디스크에서 읽어야 할 블록 수를 줄이는 것이 I/O 효율화 튜닝의 핵심 원리**

> **BCHR의 한계점**

> - 같은 블록을 반복적으로 액세스하는 형태의 Appplication일 경우 논리적인 I/O요청이 비효율적으로 많이 발생하는데도 BCHR은 높게 나타날 수 있음.
> - 작은 테이블을 자주 액세스하면 모든 블록이 메모리에서 찾아지므로 BCHR는 높겠지만 블록을 찾는 과정에서 래치를 얻어야 하므로 큰 비용을 수반.
> - 같은 블록을 여러 세션이 동시에 액세스함으로 인하여 래치 경합과 버퍼 Lock 경합까지 발생하면 메모리 I/O 비용이 디스크 I/O 이상으로 커질 수 있음.
> - BCHR가 100%라고 하더라도 논리적으로 읽어야 할 블록 수의 절대량이 많다면 반드시 SQL 튜닝을 실시해야 함.

# **네트워크, 파일시스템 캐시가 I/O 효율에 미치는 영향**


- 최근 네트워크 속도가 I/O 성능에 지대한 영향
- 이제는 데이터베이스 서버와 스토리지 간에 NAS 서버나 SAN을 통해 연결되는 아키텍쳐를 사용.
- 대용량 데이터 저장과 관리 기법에 이런 네트워크 기룻이 사용한지 오래되었고, 따라서 디스크 속도뿐 아니라 네트워크 속도까지 맞물리게 되었음.
- 최근 오라클은, 데이터베이스 서버와는 독립적으로 스토리지 자체에 CPU와 RAM을 탑재하는 하드웨어적인 솔루션을 도입.
  - 이를통해 쿼리 프로세싱 중 일부를 스토리지가 처리하게 하는 기능을 구현할 수 있음.
- 오라클 RAC는 인스턴스끼리 네트워크를 통해 캐시된 블록들을 서로 공유.
  - 메모리 I/O 성능에도 네트워크 속도가 지대한 영향을 미치게 됨.
  
> **I/O 성능에 관한 가장 확실하고 근본적인 해결책은 논리적인 블록 요청 횟수를 최소화 하는 것.**